In [ ]:
# JuliaSystems toolboxes
#Pkg.clone("https://github.com/JuliaSystems/SystemsBase.jl.git")
#Pkg.clone("https://github.com/JuliaSystems/ControlToolbox.jl.git")
#Pkg.clone("https://github.com/neveritt/IdentificationToolbox.jl.git")
Pkg.checkout("IdentificationToolbox","WIP-MIMO")
Pkg.checkout("ControlToolbox","development")

# Plotting functionality
Pkg.add("Plots")
Pkg.add("GR")

Pkg.update()

In [3]:
include("../PolynomialMatrices.jl/src/PolynomialMatrices.jl")
include("../SystemsBase.jl/src/SystemsBase.jl")
include("../ControlToolbox.jl/src/ControlToolbox.jl")
include("../IdentificationToolbox.jl/src/IdentificationToolbox.jl")
using PolynomialMatrices
using SystemsBase
using IdentificationToolbox # Use IdentificationToolbox package
using ControlToolbox        # Use ControlToolbox package
#using ODE                   # Use ODE package
using Plots                 # Use Plots frontend
gr()                        # Use GR backend


Use "local opt::Core.getfield(Optim,:OptimizationResults)" instead.


Plots.GRBackend()

In [4]:
data, header = readcsv("collected-data.csv", header=true)

(
[0.0 -0.206242 0.644553; 0.1001 -1.17932 0.586008; … ; 99.8999 -1.22364 1.99638; 100.0 -4.12476 2.20096],

AbstractString["t" "u" "y"])

In [5]:
u = data[:,2].'; y = data[:,3].';
zdata = iddata(y,u)

Discrete-time data set with 1000 samples.
Sampling time: 1.0 seconds


In [6]:
model = OE(2,2,[1]);
pem(zdata, model, zeros(6))

LoadError: + not defined for ForwardDiff.Dual{6,Float64}

### Aircraft Pitch: System Modeling

Now we will simulate the time-response of the aircraft pitch model given below:

\begin{align*}
  \begin{bmatrix}
    \dot{\alpha} \\ \dot{q} \\ \dot{\theta}
  \end{bmatrix} & = \begin{bmatrix}
    -0.313  & 56.7   & 0 \\
    -0.0139 & -0.426 & 0 \\
     0      & 56.7   & 0
  \end{bmatrix} \begin{bmatrix}
    \alpha \\ q \\ \theta
  \end{bmatrix} + \begin{bmatrix}
    0.232 \\ 0.0203 \\ 0
  \end{bmatrix} \delta \\
  y & = \begin{bmatrix}
    0 & 0 & 1
  \end{bmatrix} \begin{bmatrix}
    \alpha \\ q \\ \theta
  \end{bmatrix}
\end{align*}

In [19]:
A = [-0.313    56.7   0;
     -0.0139  -0.426  0;
      0        56.7   0];
B = [ 0.232 0.0203 0].'
C = [0 0 1]
D = 0

# generate statespace model
P = ss(A,B,C,D)

2×4 Array{RationalFunctions.RationalFunction{Val{:s},Val{:conj},Int64,Int64},2}:
 n(s̄)/d(s̄)  n(s̄)/d(s̄)  n(s̄)/d(s̄)  n(s̄)/d(s̄)
 n(s̄)/d(s̄)  n(s̄)/d(s̄)  n(s̄)/d(s̄)  n(s̄)/d(s̄)

### Analysis

Now let's see how the uncompensated open-loop system performs. Specifically, we will use the command step to analyze the open-loop step response where we have scaled the input to represent an elevator angle input ($\delta$) of 0.2 radians (11 degrees)

In [12]:
t = 0:0.01:10
step(Pₚ,t)

LoadError: MethodError: Cannot `convert` an object of type Float64 to an object of type SystemsBase.StateSpace{Val{:siso},Val{:cont},Array{Float64,2},Array{Float64,2},Array{Int64,2},Array{Int64,2}}
This may have arisen from a call to the constructor SystemsBase.StateSpace{Val{:siso},Val{:cont},Array{Float64,2},Array{Float64,2},Array{Int64,2},Array{Int64,2}}(...),
since type constructors fall back to convert methods.

The open loop response is clearly unstable. Stability of a system can be determined by examining the poles of the transfer function where the poles can be identified using the command poles as shown below.

In [7]:
isstable(Pₚ)

true

In [8]:
poles(Pₚ)

3-element Array{Complex{Float64},1}:
      -0.3695+0.885967im
      -0.3695-0.885967im
 -1.40979e-30+0.0im     

#### Closed-loop response

In order to stabilize this system and eventually meet our given design requirements, we will add a feedback controller.

In [11]:
sys_cl = feedback(Pₚ,1)
poles(sys_cl)

LoadError: MethodError: Cannot `convert` an object of type Int64 to an object of type SystemsBase.StateSpace{Val{:siso},Val{:cont},Array{Float64,2},Array{Float64,2},Array{Int64,2},Array{Int64,2}}
This may have arisen from a call to the constructor SystemsBase.StateSpace{Val{:siso},Val{:cont},Array{Float64,2},Array{Float64,2},Array{Int64,2},Array{Int64,2}}(...),
since type constructors fall back to convert methods.

In [10]:
zeros(sys_cl)

LoadError: UndefVarError: sys_cl not defined

### PID design

For a step reference of 0.2 radians, the design criteria are the following.

- Overshoot less than 10%
- Rise time less than 2 seconds
- Settling time less than 10 seconds
- Steady-state error less than 2%

### Frequency domain control design

For a step reference of 0.2 radians, the design criteria are the following.

- Overshoot less than 10%
- Rise time less than 2 seconds
- Settling time less than 10 seconds
- Steady-state error less than 2%

\begin{equation}
  P(s) = \frac{\Theta(s)}{\Delta(s)} = \frac{1.151s + 0.1774s^2}{1 + 0.739s + 0.921s^2}
\end{equation}

In [ ]:
Pₚ = tf([0, 1.151, 0.1774], [1, 0.739, 0.921])
step(0.2*Pₚ, [0,10]);

### Closed loop response
Let's now close the loop on our plant and see if that stabilizes the system. Consider the following unity feedback architecture for our system.

In [ ]:
sys_cl = feedback(Pₚ, 1)
step(sys_cl)

In [ ]:
poles(sys_cl)

Examination of the above demonstrates that the settle time requirement of 10 seconds is not close to being met. One way to address this is to make the system response faster, but then the overshoot shown above will likely become a problem. Therefore, the overshoot must be reduced in conjunction with making the system response faster. We can accomplish these goals by adding a compensator to reshape the Bode plot of the open-loop system. The Bode plot of the open-loop system indicates behavior of the closed-loop system. More specifically,

- the gain crossover frequency is directly related to the closed-loop system's speed of response, and
- the phase margin is inversely related to the closed-loop system's overshoot.

Therefore, we need to add a compensator that will increase the gain crossover frequency and increase the phase margin as indicated in the Bode plot of the open-loop system.

### Lead compensator

A type of compensator that can accomplish both of our goals is a lead compensator. A lead compensator adds positive phase to the system. Additional positive phase increases the phase margin, thus, increasing the damping. The lead compensator also generally increases the magnitude of the open-loop frequency response at higher frequencies, thereby, increasing the gain crossover frequency and overall speed of the system. Therefore, the settling time should decrease as a result of the addition of a lead compensator. The general form of the transfer function of a lead compensator is the following.

\begin{equation}
  C(s)=K \frac{Ts + 1}{\alpha Ts+1} \quad (\alpha < 1)
\end{equation}

We thus need to find $\alpha$, T and K. Typically, the gain K is set to satisfy requirements on steady-state error. Since our system is already type 1 (the plant has an integrator) the steady-state error for a step input will be zero for any value of K. Even though the steady-state error is zero, the slow tail on the response can be attributed to the fact the velocity-error constant is too small. This deficiency can be addressed by employing a value of K that is greater than 1, in other words, a value of K that will shift the magnitude plot upward. Through some trial and error, we will somewhat arbitrarily choose K = 10. Running the following code will demonstrate the effect of adding this K.

In [ ]:
K = 10;
bode(K*Pₚ)

In [ ]:
sys_cl = feedback(K*Pₚ,1);
step(0.2*sys_cl)

From examination of the above Bode plot, we have increased the system's magnitude at all frequencies and have pushed the gain crossover frequency higher. The effect of these changes are evident in the closed-loop step response shown above. Unfortunately, the addition of the K has also reduced the system's phase margin as evidenced by the increased overshoot in the system's step response. As mentioned previously, the lead compensator will help add damping to the system in order to reduce the overshoot in the step response.

Continuing with the design of our compensator, we will next address the parameter $\alpha$ which is defined as the ratio between the zero and pole. The larger the separation between the zero and the pole the greater the bump in phase where the maximum amount of phase that can be added with a single pole-zero pair is 90 degrees. The following equation captures the maximum phase added by a lead compensator as a function of $\alpha$.

$$ \sin(\phi_m)=\frac{1 - \alpha}{1 + \alpha} $$

Relationships between the time response and frequency response of a standard underdamped second-order system can be derived. One such relationship that is a good approximation for damping ratios less than approximately 0.6 or 0.7 is the following.

$$ \zeta \approx \frac{PM (degrees)}{100^{\circ}} $$

While our system does not have the form of a standard second-order system, we can use the above relationship as a starting point in our design. As we are required to have overshoot less than 10%, we need our damping ratio $\zeta$ to be approximately larger than 0.59 and thus need a phase margin greater than about 59 degrees. Since our current phase margin (with the addition of K) is approximately 10.4 degrees, an additional 50 degrees of phase bump from the lead compensator should be sufficient. Since it is known that the lead compensator will further increase the magnitude of the frequency response, we will need to add more than 50 degrees of phase lead to account for the fact that the gain crossover frequency will increase to a point where the system has more phase lag. We will somewhat arbitrarily add 5 degrees and aim for a total bump in phase of 50+5 = 55 degrees.

We can then use this number to solve the above relationship for $\alpha$ as shown below.

$$ \alpha = \frac{1 - \sin(55^{\circ})}{1 + \sin(55^{\circ})} \approx 0.10 $$

From the above, we can calculate that $\alpha$ must be less than approximately 0.10. For this value of $\alpha$, the following relationship can be used to determine the amount of magnitude increase that will be supplied by the lead compensator at the location of the maximum bump in phase.

$$ 20 \log \left( \frac{1}{\sqrt{\alpha}} \right) \approx 20 \log \left( \frac{1}{\sqrt{0.10}} \right) \approx 10 dB $$

Examining the Bode plot shown above, the magnitude of the uncompensated system equals -10 dB at approximately 6.1 rad/sec. Therefore, the addition of our lead compensator will move the gain crossover frequency from 3.49 rad/sec to approximately 6.1 rad/sec. Using this information, we can then calculate a value of T from the following in order to center the maximum bump in phase at the new gain crossover frequency in order to maximize the system's resulting phase margin.

$$ \omega_m = \frac{1}{T \sqrt{\alpha}} \Rightarrow T = \frac{1}{6.1\sqrt{.10}} \approx 0.52 $$

With the values K = 10, $\alpha$ = 0.10, and T = 0.52 calculated above, we now have a first attempt at our lead compensator. Adding the following lines to your m-file and running at the command line will generate the plot shown below demonstrating the effect of your lead compensator on the system's frequency response.

In [ ]:
s = tf([1,0],[1])
K = 10
α = 0.10
T = 0.52
Cₗ = K*tf([T, 1], [α*T, 1])
bode(Cₗ*Pₚ)

In [ ]:
sys_cl = feedback(Cₗ*Pₚ,1);
step(0.2*sys_cl)

From the above, all of our requirements are met except for the overshoot which is a bit larger than the requirement of 10%. Iterating on the above design process, we arrive at the parameters K = 10, $\alpha$ = 0.04, and T = 0.55. The performance achieved with this controller can then be verified by modifying the code as follows.

In [ ]:
s = tf([1,0],[1])
K = 10
α = 0.04
T = 0.55
Cₗ = K*tf([T, 1], [α*T, 1])
bode(Cₗ*Pₚ)

In [ ]:
sys_cl = feedback(Cₗ*Pₚ,1);
step(0.2*sys_cl)

### root locus design

In [ ]:
rp = rootlocus(P)

plot(rp, markershape = :star7, markersize = 5, markeralpha = 0.4)

animate(rp, "animation.gif")

### Controllability

In order to apply our state-space controller design techniques we need to first verify an important property, controllability. The controllability property is necessary to guarantee that we have the authority to drive the state of the system anywhere we like. This corresponds to the ability to place the closed-loop poles of the system anywhere in the complex s-plane.

The controllability Gramian is a Gramian used to determine whether or not a linear system is controllable. The controllability Gramian  is given as the 
unique solution of the Lyapunov equation

\begin{align*}
  AW_{c}+W_{c}A^{T} & = -BB^{T}
\end{align*}

is positive definite if and only if the pair $(A, B)$ is controllable.

In [ ]:
# controllability grammian (Only works for A with real eigenvalues until lyapunov solvers are updated)
# co = gram(Pₚ, :c) 
# eig(co) # check positive definiteness

Tc,c = rosenbrock(A,B) 
c == size(A,1)

`T` is a transformation to controllable realization and `c` is dimension of controllable subspace.
The controllable realization can then be calculated using

In [ ]:
Ac = Tc'*A*Tc
Bc = Tc'*B
Cc = C*Tc
Dc = D

### Pole placement

Referring back to the state-space equations at the top of the page, we see that substituting the state-feedback law $\delta$ = $\theta_r$ - K x for $\delta$ leads to the following.

\begin{align*}
  \dot{{\bf x}} & = (A - BK){\bf x} + B\theta_{r} \\
  \theta & = C{\bf x}
\end{align*}

Based on the above, matrix $A - BK$ determines the closed-loop dynamics of our system.

We know from the above that we can place the closed-loop poles of the system anywhere we would like. The question then that is left is, where should we place them? If we have a standard first- or second-order system, we then have relationships that directly relate pole locations to characteristics of the step response and can use these relations to place the poles in order to meet our given requirements. This process becomes more difficult if we have a higher-order system or zeros. With a higher-order system, one approach is to place the higher-order poles 5-10 times farther to the left in the complex plane than the dominant poles, thereby leading them to have negligible contribution to the transient response. The effect of zeros is more difficult to address using a pole-placement approach to control. Another limitation of this pole-placement approach is that it doesn't explicitly take into account other factors like the amount of required control effort.

In [ ]:
p₁ = [-10, -0.8+0.5im, -0.8-0.5im] # desired poles
K = place(Pₚ.A, Pₚ.B, p)


step


p₂ = [-1, -1, -1]






p₃ = [-0.6, -0.6+0.6im, -0.6-0.6im]
P  = [p₁, p₂, p₃]

# @gif
for p in P
    K = place(Pₚ.A, Pₚ.B, p)
    
    sys_cl = ss(A-B*K, B, C, D)
    step(sys_cl)
    
end #every 5

### Advanced user - MathProgBase interface
By following the MathProgBase interface the advanced user can use their favourite solver to solve the Poleplacement problem.

In [ ]:
using Ipopt  
K = place(A,B,p,IpoptSolver())

In [ ]:
using NLopt
K = place(A,B,p,NLoptSolver(algorithm=:LD_MMA))

### Linear quadratic regulation

We will use a technique called the Linear Quadratic Regulator (LQR) method to generate the "best" gain matrix K, without explicitly choosing to place the closed-loop poles in particular locations. This type of control technique optimally balances the system error and the control effort based on a cost that the designer specifies that defines the relative importance of minimizing errors and minimimizing control effort. In the case of the regulator problem, it is assumed that the reference is zero. Therefore, in this case the magnitude of the error is equal to the magnitude of the state. To use this LQR method, we need to define two parameters: the state-cost weighted matrix (Q) and the control weighted matrix (R). For simplicity, we will choose the control weighted matrix equal to 1 (R=1), and the state-cost matrix (Q) equal to pC'C. Employing the vector C from the output equation means that we will only consider those states in the output in defining our cost. In this case,  $\theta$ is the only state variable in the output. The weighting factor (p) will be varied in order to modify the step response. In this case, R is a scalar since we have a single input system.

In [ ]:
p = 2
Q = p*C'*C
R = 1
K = lqr(A,B,Q,R)

Note the structure of the weighting matrix Q and the resulting gain matrix K. Referring to the closed-loop state equations given above assuming a control law with non-zero reference, $\delta = \theta_{r}$ - K x $. Note that the response is scaled to model the fact that the pitch angle reference is a 0.2 radian (11 degree) step. The step response shown below should then be generated.

In [ ]:
sys_cl = ss(A-B*K, B, C, D)
step(0.2*sys_cl)

### adding precompensation

We try to make the response faster by penalizing the system error more by increasing $p$. Unlike other design methods, the full-state feedback system does not compare the output to the reference; instead, it compares all states multiplied by the control matrix $Kx$ to the reference. Thus, we should not expect the output to equal the commanded reference. To obtain the desired output, we can scale the reference input so that the output does equal the reference in steady state. This can be done by introducing a precompensator scaling factor called $\bar{N}$.

In [ ]:
p = 50;
Q = p*C'*C;
R = 1;
K = lqr(A,B,Q,R);
sys_cl = ss(A-B*K, B, C, D)

N̄ = 1/sys_cl(0)

sys_cl = ss(A-B*K,B*N̄,C,D)
step(0.2*sys_cl)

### Discrete state-space

The first step in the design of a digital control system is to generate a sampled-data model of the plant. 

In choosing a sample time, note that it is desired that the sampling frequency be fast compared to the dynamics of the system in order that the sampled output of the system captures the system's full behavior, that is, so that significant inter-sample behavior isn't missed. One measure of a system's "speed" is its closed-loop bandwidth. A good rule of thumb is that the sampling time be smaller than 1/30th of the closed-loop bandwidth frequency.
From the closed-loop Bode plot, the bandwidth frequency can be determined to be approximately 2 rad/sec (0.32 Hz). You may verify this yourself. Thus, to be sure we have a small enough sampling time, we will use a sampling time of 1/100 sec/sample. 

In [ ]:
Ts = 0.01
sys_d, x0map = c2d(sys_ss, Ts, Discretization.Bilinear())

For a system in state space form, returns the discretized system as well as a
matrix $x0map$ that transforms the initial conditions to the discrete domain by
x0_discrete = x0map*[x0, u0].

### System Identification



In [ ]:
data, header = readcsv("collected-data.csv", header=true)

u = data[:,2].'; y = data[:,3].';
zdata = iddata(y,u)

model = OE(3,3,[1]);
xb, pe = IdentificationToolbox._morsm(zdata, model)

idmodel = pem(zdata, model, xb[1:6], IdOptions(estimate_initial=false,autodiff=true,iterations=100))

In [ ]:
idmodel.in